In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import bcolz
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import math
import pickle
from tqdm import tqdm
from torch.autograd import Variable
# plotting
import matplotlib
matplotlib.use('Agg')
from vad import batchData
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

"""
Set seed #
"""
seed = 1337

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def loadDataset(path = '../Datasets/Reviews/dataset_ready.pkl'):
    return pickle.load(open(path, 'rb'))

In [2]:
print("Loading parameters..", end=" ")
hiddenSize = 512
latentSize = 400
batchSize  = 64
iterations = 1
learningRate = 0.0001
bidirectionalEncoder = True
# device = "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Done.")

print("Loading dataset..", end=" ")
dataset = loadDataset()
# setup store parameters
id2word = dataset['id2word']
word2id = dataset['word2id']
weightMatrix = dataset['weights']
train = dataset['train']
validation = dataset['validation']
cutoff = dataset['cutoff']
paddingID = word2id['<pad>']
print("Done.")

print("Converting dataset weights into tensors..", end=" ")
# convert dataset into tensors
weightMatrix = torch.tensor(weightMatrix, dtype=torch.float)
print("Done.")

# batching data
print("Batching Data..",end=" ")

# shuffle data rows
random.shuffle(train)
random.shuffle(validation)

# trainx = [x[0] for x in train]
# trainy = [x[1] for x in train]
valx = [x[0] for x in validation]
valy = [x[1] for x in validation]

# shuffle data row

# trainx = batchData(trainx, paddingID, device, batchSize, cutoff)
# trainy = batchData(trainy, paddingID, device, batchSize, cutoff)
valx = batchData(valx, paddingID, device, batchSize, cutoff)
valy = batchData(valy, paddingID, device, batchSize, cutoff)

# traindata = (trainx, trainy)
valdata= (valx, valy)
print("Done.")

# setup variables for model components initialisation
maxReviewLength = cutoff
vocabularySize = len(id2word)
embeddingDim = weightMatrix.shape[1]
embedding_shape = weightMatrix.shape

Loading parameters.. Done.
Loading dataset.. Done.
Converting dataset weights into tensors.. Done.
Batching Data.. Done.


In [3]:
from vad import Encoder, Decoder, Attn, Backwards, Prior, Inference, loss_function

print("Initialising model components..", end=" ")

modelEncoder = Encoder(weightMatrix, vocabularySize,
                       paddingID, hiddenSize, bidirectionalEncoder).to(device)
# modelAttention = Attention(maxLength=maxReviewLength).to(device)
modelAttention = Attn(hiddenSize, bidirectionalEncoder).to(device)
modelBackwards = Backwards(weightMatrix, vocabularySize,
                           paddingID, hiddenSize, bidirectionalEncoder).to(device)
modelInference = Inference(
    hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelPrior = Prior(hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelDecoder = Decoder(weightMatrix, vocabularySize,
                       paddingID, batchSize, maxReviewLength, hiddenSize, latentSize, bidirectionalEncoder).to(device)
criterion = nn.NLLLoss(ignore_index=paddingID)
print("Done.")

filepath = "attempt_1/"
# load models
print("Loading model weights..", end=" ")
modelEncoder.load_state_dict(torch.load(os.path.join(filepath,'encoder.pth')))
modelAttention.load_state_dict(torch.load(os.path.join(filepath,'attention.pth')))
modelBackwards.load_state_dict(torch.load(os.path.join(filepath,'backwards.pth')))
modelInference.load_state_dict(torch.load(os.path.join(filepath,'inference.pth')))
modelPrior.load_state_dict(torch.load(os.path.join(filepath,'prior.pth')))
modelDecoder.load_state_dict(torch.load(os.path.join(filepath,'decoder.pth')))
print("Done.")

Initialising model components.. Done.
Loading model weights.. Done.


In [4]:
def evaluate(x,
             xLength,
             y,
             yLength,
             encoder,
             attention,
             backwards,
             inference,
             prior,
             decoder,
             criterion
            ):
    
    loss = 0
    
    # initalise input and target lengths
    inputLength = x[0].size(0)
    targetLength = y[0].size(0)
    batchSize = x.shape[0]

    # set up encoder computation
    encoderHidden = encoder.initHidden(batchSize).to(device)
    backwardHidden = backwards.initHidden(batchSize).to(device)
    
    # set up encoder outputs
    encoderOutputs, encoderHidden = encoder(x, encoderHidden, xLength)

    # compute backwards outputs
    backwardOutput, backwardHidden = backwards(torch.flip(
        y, [0, 1]), yLength, backwardHidden)

    # set up the variables for decoder computation
    decoderInput = torch.tensor([[word2id["<sos>"]]] * batchSize, dtype=torch.long, device=device)
    
    decoderHidden = encoderHidden[-1]
    decoderOutput = None
    decoderOutputs = []
    
    # Run through the decoder one step at a time. This seems to be common practice across
    # all of the seq2seq based implementations I've come across on GitHub.
    for t in range(yLength[0]):
        # get the context vector c
        c = attention(encoderOutputs, decoderHidden)
        
        # compute the inference layer
        z_infer, infer_mu, infer_logvar = inference(decoderHidden, c, backwardOutput[:,t])
        
        # compute the prior layer
        z_prior, prior_mu, prior_logvar = prior(decoderHidden, c)
    
        # compute the output of each decoder state
        DecoderOut = decoder(decoderInput, c, z_prior, decoderHidden)
        
        # update variables
        decoderOutput, decoderHidden = DecoderOut
        decoderOutputs.append(decoderOutput)
#         seqloss = loss_function(decoderOutput, y[:, t], infer_mu, infer_logvar, prior_mu, prior_logvar, criterion)
#         loss += seqloss
        loss = "unk"

        # feed this output to the next input
        decoderInput = y[:,t]
        decoderHidden = decoderHidden.squeeze(0)
        
#     loss = loss.item()/targetLength
        
    return decoderOutputs, loss

In [5]:
def evaluateModel(batched_data,
                   encoder,
                   backward,
                   attention,
                   inference,
                   prior,
                   decoder,
                   criterion,
                   id2word,
                  ):
    
    encoder.eval()
    backward.eval()
    attention.eval()
    inference.eval()
    prior.eval()
    decoder.eval()
    numbatches = len(batched_data)
    for batch in tqdm(range(0,3)):
        # each batch is composed of the 
        # reviews, and a sentence length.
        x, xLength = batched_data[0][batch][0], batched_data[0][batch][1]
        y, yLength = batched_data[1][batch][0], batched_data[1][batch][1]
        
        outputs, losses = evaluate(x,
                                   xLength,
                                   y,
                                   yLength,
                                   encoder,
                                   attention,
                                   backward,
                                   inference,
                                   prior,
                                   decoder,
                                   criterion)
        
        print(y.shape)
        reference = convertRealID2Word(id2word,y)
        words = convertDecoderID2Word(id2word,outputs)

        for i in range(len(reference)):
            print("REFERENCE:", reference[i])
            print("MODEL:    ",words[i])
            print()
        
        print("LOSS:", losses)
#         break

def convertRealID2Word(id2word, y):
    entries = [[id2word[x.item()] for x in y[entry].cpu()] for entry in range(len(y))]
    for i in range(len(entries)):
        entries[i] = " ".join(entries[i][12:])
        entries[i] = entries[i].replace("<pad> ", "")
#         entries[i] = entries[i][43:]
    return entries
        
def convertDecoderID2Word(id2word, outputs):
    entries = []
    for batch_line in outputs:
        entry = [torch.argmax(batch_line[i]).cpu().item() for i in range(len(batch_line))]
        entries.append([id2word[i] for i in entry])
     
    words = []
    for i in range(len(outputs[0])):
#         line = "".join([entries[j][i] for j in range(0,12)])

        line = " ".join([entries[j][i] for j in range(12,len(entries))])
        line = line.replace(" ' ", "'")
        line = line.replace(" ) ", ")")
        line = line.replace(" . ", ".")
        line = line.replace(" , ", ", ")
        line = line.replace(" ( ", "(")
        words.append(line)
    return words




In [6]:
outputs = evaluateModel(valdata,
                   modelEncoder,
                   modelBackwards,
                   modelAttention,
                   modelInference,
                   modelPrior,
                   modelDecoder,
                   criterion,
                   id2word
                  )

 33%|███▎      | 1/3 [00:04<00:08,  4.50s/it]

torch.Size([64, 60])
REFERENCE: but seems very erratic along with the toshiba app and as i said i don ' t use it normally in the home but transfer directly with the card itself . the ideal app would allow me to select and mail multiple pix with only one step but
MODEL:     rating_5.0 rating_5.0 a enough b to i and <unk> b i.<unk> this <unk> this.i <unk> this to speaker t ,.this i <unk>.of i...b t b model p.i i...i.b

REFERENCE: some of the dials of the x100 , it ' s not really that type of camera . i cannot recommend this enough for anyone looking for a relatively compact ( not <unk> ) camera that takes great pictures , as long as you aren ' t looking
MODEL:     1 1 with x b with of b of of isn.i, of.with of.<unk> of 1 a if isn into b.<unk>.t.i b.b of not <unk> to your.i.this this isn i

REFERENCE: follow their video , and the install goes just smooth ! clean ( thoroughly ) using the included wipe , use the dust removal sticker , and then , carefully apply the protector ( holding by th

 67%|██████▋   | 2/3 [00:08<00:04,  4.48s/it]

torch.Size([64, 60])
REFERENCE: and incapable of powering the unit . backlight of the screen is anemic compared to the 700 , but it sips power in comparison . if you already have a decent notebook or desktop , probably better to stick with them , but for a tablet , this
MODEL:     rating_5.0 rating_5.0 i that your.b i b with of <unk>.b b i.b of of of i i.t <unk> i this i of <unk> decent works <unk> to of i a p i f x so of f <unk> f so

REFERENCE: , not worry about missing the shot because i have the wrong lens on the camera and minimize the number of times i ' m swapping lenses and getting more gunk onto my sensor . after real life usage of the <unk> both on and off a tripod
MODEL:     polarity_-0.1 polarity_-0.1 polarity_-0.1 to 1 a.i of <unk> of need t x <unk> <unk> b i b.b with of ? purchase <unk>.b i expect i.....i <unk> b b with of b b <unk> <unk> thought <unk>

REFERENCE: new iphone 5 . installation was easy enough although i couldn ' t believe that they put a sticker on the insi

100%|██████████| 3/3 [00:13<00:00,  4.44s/it]

torch.Size([64, 60])
REFERENCE: the ipad screen was , i still got tons of air bubbles that couldn ' t be removed ( bubbles supposedly caused by dust particles ) . also , in spite of the company ' s insistence that they stand behind their products , my four + emails
MODEL:     polarity_-0.1 polarity_-0.1 to a i of <unk> i b of with i to, i isn i.deep <unk> b i.to...<unk> i of 5 i <unk> <unk> and...of <unk> <unk> b i b of.part .

REFERENCE: do not have access to the physical router then plug this device into the wall , connect to it directly via wifi via it ' s own web enabled interface and walk through the simple wizard to pick and authenticate with the wifi signal of your choosing .
MODEL:     at, i of t i.to b i.a t.. i so i i.i to.to this isn.to.to.i t.. b b i i i.... i with of i

REFERENCE: <unk> recommends , before adding this monitor . i ' m also using a logitech 520 wireless keyboard and mouse , samsung dvd usb drive , logitech speakers , and various other accessories via my <unk